## Input the Parameters of the Starlette-SLR GEODYN run:

In [2]:
import pandas as pd
import numpy as np

In [14]:
ll /data/runs_geodyn/iss/results/msis2/msis2_accelon/IIEOUT/


total 346644
-rw-rw-r--. 1 m_geodyn 354960125 Feb 17 22:42 iss190420_2dy.goco05s


In [15]:
ll /data/analysis/


total 355528
-rw-rw-r--. 1 m_geodyn   4319055 Feb 18 17:34 DEN_iss190420_2dy.goco05s
-rw-r--r--. 1 m_geodyn    447595 Jan 13 16:47 geodynBinary_to_Python
-rw-rw-r--. 1 m_geodyn 354960125 Feb 18 17:35 IIEOUT_iss190420_2dy.goco05s
-rw-rw-r--. 1 m_geodyn   4319055 Feb 18 17:33 iss190420_2dy.goco05s
drwxr-xr-x. 5 m_geodyn       137 Feb 18 16:57 ISS_GPS_analysis/
drwxrwxr-x. 2 m_geodyn       131 Dec 15 19:00 __pycache__/
drwxr-xr-x. 3 m_geodyn       224 Jan 14 16:19 scratch_codes/
drwxr-xr-x. 6 m_geodyn      4096 Feb 17 21:01 starlette_analysis/
drwxr-xr-x. 6 m_geodyn       216 Feb 17 20:58 util_funcs/


In [16]:


########################
# INPUT PARAMETERS:
########################
sat_file = 'iss'
arc = '190420_2dy'
grav_id ='goco05s' 
local_path = '/data/analysis/ISS_GPS_analysis/'
SAT_ID = 9806701
accel_card = 'accelon'


#######################################
# PATH TO DENSITY MODEL RUN of Choice:
#######################################
msis2_model = 'msis2'
path_to_msis2 = '/data/runs_geodyn/'+sat_file+'/results/'+ msis2_model+'/'+ msis2_model+ '_' +accel_card+'/'



### Call the prep-function that loads the data

This may take a few minutes to load.

#### MSIS2

In [17]:
# %load_ext autoreload
# %autoreload 2

# import sys  
# sys.path.insert(0, '/data/analysis/util_funcs/py_geodynreader_gps/')

# from a_ReadISS import ReadISS

# AdjustedParams, Trajectory, Density, Resids = ReadISS(arc, 
#                                                            sat_file,
#                                                            grav_id, 
#                                                            local_path, 
#                                                            path_to_msis2,
#                                                            True)

In [20]:
# WONKY INPUTS:
path_to_data = path_to_msis2
AccelStatus = True


import sys  
sys.path.insert(0, '/data/analysis/notebooks/util_funcs/py_geodynreader_gps/')
SAT_ID = 9806701
sat = sat_file
file_name =  sat + arc + '.'+ grav_id
print('The base file name for this arc is:',file_name,'\n' )
import os

iieout_file  = '/data/analysis/IIEOUT_'+ file_name   #path_to_data + 'IIEOUT/'+ file_name
density_file = '/data/analysis/DEN_'+ file_name  
print(iieout_file)
print(density_file)

# from b_ReadISS import Save_AdjustedParameters_ISS
# SatMain_AdjustedParams = Save_AdjustedParameters_ISS(SAT_ID, iieout_file, AccelStatus)


The base file name for this arc is: iss190420_2dy.goco05s 

/data/analysis/IIEOUT_iss190420_2dy.goco05s
/data/analysis/DEN_iss190420_2dy.goco05s


In [21]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import os.path
import linecache
densityfil = density_file

In [29]:
def read_density_file_ISS(densityfil):


    DEN_csv = pd.read_csv(densityfil, 
                        skiprows = 1, 
                        names = ['Elapsed Secs',
                                'YYMMDD',
                                'HHMMSS',
                                'Lat',
                                'Lon',
                                'Height (meters)',
                                'rho (kg/m**3)',
                                'delta_rho (kg/m**3/m)',
                                'X',
                                'Y',
                                'Z',
                                'XDOT',
                                'YDOT',
                                'ZDOT',
                              ],
                        sep = '\s+',
                        )

    #The below takes the Double precision format from Fortran (D) and puts an 
    #E in the string to be interpeted as a float by python
    DEN_df = pd.DataFrame(DEN_csv)
    fix_D_decimal_to_E = []
    fix_D_decimal_to_E2 = []

    for i,val in enumerate(DEN_df['rho (kg/m**3)']):
        val2 = DEN_df['delta_rho (kg/m**3/m)'][i]

        list_val = list(val)
        list_val2 = list(val2)

        indx = list(val).index('D')
        indx2 = list(val2).index('D')

        list_val[indx] = 'E'
        list_val2[indx2] = 'E'

        list_val = "".join(list_val)
        list_val2 = "".join(list_val2)

        val_float = np.float(list_val)
        val_float2 = np.float(list_val2)

        fix_D_decimal_to_E.append(val_float)
        fix_D_decimal_to_E2.append(val_float2)


    DEN_df['rho (kg/m**3)'] = fix_D_decimal_to_E
    DEN_df['delta_rho (kg/m**3/m)'] = fix_D_decimal_to_E2

    # Now we must correct the formatting of the HoursMinutesSeconds column
    timeHHMMSS = [] 
    for i,val in enumerate(DEN_df['HHMMSS'].values.astype(int)):
        # print(len(str(val)))
        if len(str(val)) == 1:
            timehhmmss_val = '00000'+ str(val)
            timeHHMMSS.append(timehhmmss_val)
        elif len(str(val)) == 2:
            timehhmmss_val = '0000'+ str(val)
            timeHHMMSS.append(timehhmmss_val)
        elif len(str(val)) == 3:
            timehhmmss_val = '000'+ str(val)
            timeHHMMSS.append(timehhmmss_val)
        elif len(str(val)) == 4:
            timehhmmss_val = '00'+ str(val)
            timeHHMMSS.append(timehhmmss_val)
        elif len(str(val)) == 5:
            timehhmmss_val = '0'+ str(val)
            timeHHMMSS.append(timehhmmss_val)
        else:
            timeHHMMSS.append(str(val))
    DEN_df['timeHHMMSS'] = timeHHMMSS

    # Here we split up the years, months, and days.
    # we add a '20' in front of the year

    year  = np.zeros(np.size(DEN_df['YYMMDD'].values))
    month = np.zeros(np.size(DEN_df['YYMMDD'].values))
    day   = np.zeros(np.size(DEN_df['YYMMDD'].values))
    hours  = np.zeros(np.size(DEN_df['YYMMDD'].values))
    minutes = np.zeros(np.size(DEN_df['YYMMDD'].values))
    secs  = np.zeros(np.size(DEN_df['YYMMDD'].values))
    start2 = time.time()
    total_num = (np.size(DEN_df['YYMMDD']))

    i = 0
    for index, row in DEN_df.iterrows():
        # print()
        val = DEN_df['YYMMDD'][index].astype(int).astype(str)
        year[i] = ('20' + val[:2])
        month[i] = (val[2:4])
        day[i] = (val[4:])
        hours[i] = str(DEN_df['timeHHMMSS'][index])[:2]
        minutes[i] = str(DEN_df['timeHHMMSS'][index])[2:4]
        secs[i] = str(DEN_df['timeHHMMSS'][index])[4:]
    #     print('year', year[i])
    #     print('month', month[i])
    #     print('day', day[i] )
    #     print('hours', hours[i] )
    #     print('minu',  minutes[i])
    #     print('sec', secs[i])
        i += 1
    # end2 = time.time()
    # elapsed2 = end2 - start2
    # print(i,'/',total_num ,' Elapsed: ',elapsed2     ,   sep ='')

    DEN_df['year']  = year
    DEN_df['month'] = month
    DEN_df['day']   = day
    DEN_df['hours']  = hours
    DEN_df['minutes'] = minutes
    DEN_df['secs']  = secs


    year= list(map(int, DEN_df['year'].values))
    month= list(map(int, DEN_df['month'].values))
    day= list(map(int, DEN_df['day'].values))
    hour= list(map(int, DEN_df['hours'].values))
    minute = list(map(int, DEN_df['minutes'].values))
    second = list(map(int, DEN_df['secs'].values))

    DATE = list(map(datetime, year,month, day, hour,minute,second ))

    DEN_df['Date']  = DATE



    return(DEN_df)




In [30]:
DEN_df

,Elapsed Secs,YYMMDD,HHMMSS,Lat,Lon,Height (meters),rho (kg/m**3),delta_rho (kg/m**3/m),X,Y,...,YDOT,ZDOT,timeHHMMSS,year,month,day,hours,minutes,secs,Date
0,210.0,190420,210439,51.6232,205.1851,415666.944,9.358100e-13,0.0,4165351.995,713181.457,...,7441.052494,414.258339,210439,2019.0,4.0,20.0,21.0,4.0,39.0,2019-04-20 21:04:39
1,220.0,190420,210449,51.6749,206.1802,415670.746,9.443125e-13,0.0,4147047.339,787544.941,...,7431.486289,346.441408,210449,2019.0,4.0,20.0,21.0,4.0,49.0,2019-04-20 21:04:49
2,230.0,190420,210459,51.7174,207.1774,415672.401,9.528238e-13,0.0,4128214.028,861808.026,...,7420.972680,278.580182,210459,2019.0,4.0,20.0,21.0,4.0,59.0,2019-04-20 21:04:59
3,240.0,190420,210509,51.7508,208.1764,415671.903,9.613391e-13,0.0,4108854.463,935961.242,...,7409.513001,210.683303,210509,2019.0,4.0,20.0,21.0,5.0,9.0,2019-04-20 21:05:09
4,250.0,190420,210519,51.7749,209.1767,415669.249,9.698538e-13,0.0,4088971.110,1009995.138,...,7397.108708,142.759414,210519,2019.0,4.0,20.0,21.0,5.0,19.0,2019-04-20 21:05:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22143,106910.0,190422,24259,20.8315,192.1807,409365.110,1.311421e-12,0.0,827029.384,6292209.195,...,2681.541302,-5365.296662,024259,2019.0,4.0,22.0,2.0,42.0,59.0,2019-04-22 02:42:59
22144,106920.0,190422,24309,20.3440,192.5967,409315.310,1.310521e-12,0.0,779197.866,6318622.313,...,2601.025898,-5395.681261,024309,2019.0,4.0,22.0,2.0,43.0,9.0,2019-04-22 02:43:09
22145,106930.0,190422,24319,19.8553,193.0098,409267.612,1.330625e-12,0.0,731266.853,6344228.604,...,2520.177632,-5425.375137,024319,2019.0,4.0,22.0,2.0,43.0,19.0,2019-04-22 02:43:19
22146,106940.0,190422,24329,19.3654,193.4202,409222.059,1.329234e-12,0.0,683242.466,6369024.791,...,2439.006890,-5454.374457,024329,2019.0,4.0,22.0,2.0,43.0,29.0,2019-04-22 02:43:29


1 5041144
1 5044284
1 5051204
1 5345214
1 5347224
1 5356164
1 5459194
1 5460234
1 5461024
1 5553175
1 5652315
1 5658125
1 5755155
1 5757295
1 5848075
1 5950055
1 6062256
1 6163016
1 6265246
1 6366276
1 6464306
1 6467066
1 6468096
1 6469036
1 6571266
1 6572086
1 6573106
1 6670326
1 6834183
1 6974045
1 9743134
1 9946114
1 9806701
2 5041144
2 5044284
2 5051204
2 5345214
2 5347224
2 5356164
2 5459194
2 5460234
2 5461024
2 5553175
2 5652315
2 5658125
2 5755155
2 5757295
2 5848075
2 5950055
2 6062256
2 6163016
2 6265246
2 6366276
2 6464306
2 6467066
2 6468096
2 6469036
2 6571266
2 6572086
2 6573106
2 6670326
2 6834183
2 6974045
2 9743134
2 9946114
2 9806701
3 5041144
3 5044284
3 5051204
3 5345214
3 5347224
3 5356164
3 5459194
3 5460234
3 5461024
3 5553175
3 5652315
3 5658125
3 5755155
3 5757295
3 5848075
3 5950055
3 6062256
3 6163016
3 6265246
3 6366276
3 6464306
3 6467066
3 6468096
3 6469036
3 6571266
3 6572086
3 6573106
3 6670326
3 6834183
3 6974045
3 9743134
3 9946114
3 9806701
4 5041144


5041144
1
5044284
1
5051204
1
5345214
1
5347224
1
5356164
1
5459194
1
5460234
1
5461024
1
5553175
1
5652315
1
5658125
1
5755155
1
5757295
1
5848075
1
5950055
1
6062256
1
6163016
1
6265246
1
6366276
1
6464306
1
6467066
1
6468096
1
6469036
1
6571266
1
6572086
1
6573106
1
6670326
1
6834183
1
6974045
1
9743134
1
9946114
1
9806701
1
5041144
2
5044284
2
5051204
2
5345214
2
5347224
2
5356164
2
5459194
2
5460234
2
5461024
2
5553175
2
5652315
2
5658125
2
5755155
2
5757295
2
5848075
2
5950055
2
6062256
2
6163016
2
6265246
2
6366276
2
6464306
2
6467066
2
6468096
2
6469036
2
6571266
2
6572086
2
6573106
2
6670326
2
6834183
2
6974045
2
9743134
2
9946114
2
9806701
2
5041144
3
5044284
3
5051204
3
5345214
3
5347224
3
5356164
3
5459194
3
5460234
3
5461024
3
5553175
3
5652315
3
5658125
3
5755155
3
5757295
3
5848075
3
5950055
3
6062256
3
6163016
3
6265246
3
6366276
3
6464306
3
6467066
3
6468096
3
6469036
3
6571266
3
6572086
3
6573106
3
6670326
3
6834183
3
6974045
3
9743134
3
9946114
3
9806701
3
5041144
4


6974045
4
9743134
4
9946114
4
9806701
4
5041144
5
5044284
5
5051204
5
5345214
5
5347224
5
5356164
5
5459194
5
5460234
5
5461024
5
5553175
5
5652315
5
5658125
5
5755155
5
5757295
5
5848075
5
5950055
5
6062256
5
6163016
5
6265246
5
6366276
5
6464306
5
6467066
5
6468096
5
6469036
5
6571266
5
6572086
5
6573106
5
6670326
5
6834183
5
6974045
5
9743134
5
9946114
5
9806701
5
5041144
6
5044284
6
5051204
6
5345214
6
5347224
6
5356164
6
5459194
6
5460234
6
5461024
6
5553175
6
5652315
6
5658125
6
5755155
6
5757295
6
5848075
6
5950055
6
6062256
6
6163016
6
6265246
6
6366276
6
6464306
6
6467066
6
6468096
6
6469036
6
6571266
6
6572086
6
6573106
6
6670326
6
6834183
6
6974045
6
9743134
6
9946114
6
9806701
6
5041144
1
5044284
1
5051204
1
5345214
1
5347224
1
5356164
1
5459194
1
5460234
1
5461024
1
5553175
1
5652315
1
5658125
1
5755155
1
5757295
1
5848075
1
5950055
1
6062256
1
6163016
1
6265246
1
6366276
1
6464306
1
6467066
1
6468096
1
6469036
1
6571266
1
6572086
1
6573106
1
6670326
1
6834183
1
6974045
1


4
6464306
4
6467066
4
6468096
4
6469036
4
6571266
4
6572086
4
6573106
4
6670326
4
6834183
4
6974045
4
9743134
4
9946114
4
9806701
4
5041144
5
5044284
5
5051204
5
5345214
5
5347224
5
5356164
5
5459194
5
5460234
5
5461024
5
5553175
5
5652315
5
5658125
5
5755155
5
5757295
5
5848075
5
5950055
5
6062256
5
6163016
5
6265246
5
6366276
5
6464306
5
6467066
5
6468096
5
6469036
5
6571266
5
6572086
5
6573106
5
6670326
5
6834183
5
6974045
5
9743134
5
9946114
5
9806701
5
5041144
6
5044284
6
5051204
6
5345214
6
5347224
6
5356164
6
5459194
6
5460234
6
5461024
6
5553175
6
5652315
6
5658125
6
5755155
6
5757295
6
5848075
6
5950055
6
6062256
6
6163016
6
6265246
6
6366276
6
6464306
6
6467066
6
6468096
6
6469036
6
6571266
6
6572086
6
6573106
6
6670326
6
6834183
6
6974045
6
9743134
6
9946114
6
9806701
6
5041144
1
5044284
1
5051204
1
5345214
1
5347224
1
5356164
1
5459194
1
5460234
1
5461024
1
5553175
1
5652315
1
5658125
1
5755155
1
5757295
1
5848075
1
5950055
1
6062256
1
6163016
1
6265246
1
6366276
1
6464306
